In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from environment import *

with open("setting.yaml") as yaml_file:

    SETTING = yaml.load(yaml_file)

PATH = make_path_dict(SETTING)

In [ ]:
feature_data = SETTING["feature_data"]

if feature_data is None:

    feature_data = {}

for data_name, data_dict in feature_data.items():

    print(data_name)

    data_dict["df"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
sample_data = SETTING["sample_data"]

if sample_data is None:

    sample_data = {}

for data_name, data_dict in sample_data.items():

    print(data_name)

    data_dict["df"] = pd.read_csv(data_dict["file_path"], sep="\t", index_col=0)

In [ ]:
w = pd.read_csv(PATH["w.tsv"], sep="\t", index_col=0)

w.columns.name = "Factor"

h = pd.read_csv(PATH["h.tsv"], sep="\t", index_col=0)

h.columns.name = SETTING["sample_alias"]

In [ ]:
w__cluster_x_element = pd.read_csv(
    PATH["w|cluster_x_column.tsv"], sep="\t", index_col=0
)

w__cluster_x_element.columns.name = w.index.name

h__cluster_x_element = pd.read_csv(
    PATH["h|cluster_x_column.tsv"], sep="\t", index_col=0
)

h__cluster_x_element.columns.name = h.columns.name

In [ ]:
if SETTING["h_summary"] is not None:

    for factor_name, data_name_indices in SETTING["h_summary"].items():

        data_dicts = {}

        score_moe_p_value_fdr_dicts = {}

        factor_alias = data_name_indices.pop("alias", "Alias")

        for data_name, indices in data_name_indices.items():

            if data_name != "alias":

                data_dicts[data_name] = {
                    "df": sample_data[data_name]["df"].loc[indices],
                    "type": sample_data[data_name]["type"],
                }

                score_moe_p_value_fdr_dicts[data_name] = pd.read_csv(
                    "{}/{}/{}.tsv".format(
                        PATH["h|match/"],
                        factor_name,
                        ccal.make_file_name_from_str(data_name),
                    ),
                    sep="\t",
                    index_col=0,
                )

        ccal.make_summary_match_panel(
            h.loc[factor_name],
            data_dicts,
            score_moe_p_value_fdr_dicts,
            plot_only_columns_shared_by_target_and_all_data=SETTING[
                "plot_only_columns_shared_by_target_and_all_data"
            ],
            plot_std=SETTING["plot_std"],
            title=dict(text=factor_alias),
            xaxis_kwargs=dict(ticks="", showticklabels=False),
            html_file_path="{}/{}".format(PATH["h|summary/"], factor_name),
        )